# Statistical analysis

_Michał Denkiewicz 2024_

In [1]:
import os
import string
import functools
import numpy as np
import pandas as pd
import networkx as nx

import matplotlib
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
import statsmodels.stats.multitest
import networkx as nx
import pyranges as pr

from itertools import product
from functools import partial

from knots_tools import *

# Number of minors detected

In [ ]:
DATA_DIR = './data'

ccds = pd.read_csv(
     os.path.join(DATA_DIR, 'all_ccds.csv'),
     index_col=CCD_INDEX_NAMES,
     dtype=CCD_INDEX_DTYPES
)
ccds.info()
ccds.head()

In [ ]:
minors = pd.read_csv(
    os.path.join(DATA_DIR, 'all_minors.csv'),
    index_col=CCD_INDEX_NAMES + ['minor_id'],
    dtype=CCD_INDEX_DTYPES
)
minors.info()
minors.head()

In [ ]:
minors.groupby(['dataset'], observed=True).size()

In [ ]:
minors.groupby(['dataset', 'chromosome'], observed=True).size().unstack('dataset', fill_value=0)

In [ ]:
def pretty_formated_percent(x, precision=1):
    m = x.mean()
    if np.isnan(m):
        m = 0.0
    return f'{100.0 * m:.{precision}f}%'

ccds.has_minors.groupby(['dataset'], observed=True).agg(['sum', pretty_formated_percent]).rename(columns={'sum': 'N', 'pretty_formated_percent': 'percent'})

In [ ]:
_tab = ccds.has_minors.groupby(['dataset', 'chromosome'], observed=True).agg(['sum', pretty_formated_percent]).rename(columns={'sum': 'N', 'pretty_formated_percent': 'percent'})
_tab.columns.names = ['var']
_tab = _tab.unstack(['dataset'], fill_value=0)
_tab = _tab.reorder_levels(['dataset', 'var'], axis=1).sort_index(axis=1)
_tab

# Node / edge characteristics

In [ ]:
nodes = pd.read_csv(
    os.path.join(DATA_DIR, 'all_nodes.csv'),
    index_col=CCD_INDEX_NAMES + ['node_id'],
    dtype=CCD_INDEX_DTYPES
)
nodes.info()
nodes.head()

In [ ]:
edges = pd.read_csv(
    os.path.join(DATA_DIR, 'all_edges.csv'),
    index_col=CCD_INDEX_NAMES + ['edge_id'],
    dtype=CCD_INDEX_DTYPES
)
edges.info()
edges.head()

In [ ]:
nodes.in_minor.groupby(['dataset'], observed=True).agg(['sum', pretty_formated_percent, 'count']).rename(columns={'sum': 'N', 'pretty_formated_percent': 'percent', 'count': 'total'})

In [ ]:
edges.in_minor.groupby(['dataset'], observed=True).agg(['sum', pretty_formated_percent, 'count']).rename(columns={'sum': 'N', 'pretty_formated_percent': 'percent', 'count': 'total'})

## Graph measures

In [ ]:
ccds.head()

In [ ]:
measure_labels = {
    'n_nodes': 'No. Nodes',
    'n_edges': 'No. Edges',
    'density': 'Density',
    'degree_mean': 'Avg. Degree',
    'closeness_mean': 'Avg. Closeness',
    'betweenness_mean': 'Avg. Betweenness'
}

data_long = ccds.set_index('has_minors', append=True).loc[:, list(measure_labels.keys())].stack().to_frame('value')
data_long.index.names = data_long.index.names[:-1] + ['measure']
data_long

In [ ]:
def do_test(df, var, by, which='med'):
    (key0, g0), (key1, g1) = list(df[var].groupby(df[by]))
    assert key0 == False and key1 == True
    assert which.lower() in ('med', 't', 'u')
    if which.lower() != 't':
        m0 = g0.median()
        m1 = g1.median()
        mg = df[var].median()
    else:
        m0 = g0.mean()
        m1 = g1.mean()
        mg = df[var].mean()
    try:
        match which.lower():
            case 'med':
                stat, p, *_ = scipy.stats.median_test(g0, g1)
            case 't':
                stat, p, *_ = scipy.stats.ttest_ind(g0, g1, equal_var=False)
            case 'u':
                stat, p, *_ = scipy.stats.mannwhitneyu(g0, g1, use_continuity=True, alternative='two-sided')
    except Exception as ex:
        stat, p, mg = np.nan, np.nan, mg
    return {'stat': stat, 'pvalue': p, 'm0': m0, 'm1': m1, 'gm': mg, 'n0': len(g0), 'n1': len(g1)}

stats_tab = pd.DataFrame.from_dict(
    data_long.reset_index().groupby(['dataset', 'measure'], observed=False).apply(do_test, 'value', 'has_minors', include_groups=False).to_dict(),
    orient='index',    
)
stats_tab.index.names = ['dataset', 'measure']
stats_tab['dir'] = np.where(stats_tab['m0'] > stats_tab['m1'], '>', '<')
_, stats_tab['pcorr'], *_ = statsmodels.stats.multitest.multipletests(stats_tab['pvalue'], method='holm')
stats_tab['sig'] = pd.cut(stats_tab['pcorr'], bins=[-1, 0.001, 0.01, 0.05, 1], labels=['***', '**', '*', 'ns'], right=True)
stats_tab

In [ ]:
pretty_stats_tab = stats_tab.reset_index().replace({'measure': measure_labels, 'dataset': {'GM12878': 'GM12878(IS)', 'GM12878lr': 'GM12878(LR)'}}).set_index(['dataset', 'measure'])
pretty_stats_tab

# Multiplexes (ChIA-Drop)

In [16]:
fragments_df  = pd.read_csv(os.path.join(DATA_DIR, 'chiadrop_fragments.csv'))
segments_df = pd.read_csv(os.path.join(DATA_DIR, 'all_minor_segments.csv'))

In [17]:
fragments_pr = pr.PyRanges(fragments_df.rename(columns={'chromosome': 'Chromosome', 'frag_start': 'Start', 'frag_end': 'End'}))

In [18]:
ccds_gm12878 = ccds.loc['GM12878']
ccds_pr = pr.PyRanges(ccds_gm12878.reset_index().loc[:, ['ccd_id', 'chromosome', 'start', 'end', 'has_minors', 'length']].rename(columns={'chromosome': 'Chromosome', 'start': 'Start', 'end': 'End'}))

In [19]:
segments_gm12878 = segments_df.loc[segments_df.dataset == 'GM12878', :]
segments_pr = pr.PyRanges(segments_gm12878.rename(columns={'chromosome': 'Chromosome', 'start': 'Start', 'end': 'End'}))

In [ ]:
ccd2frag = ccds_pr.join(fragments_pr, suffix='_frag', how='left').df
ccd2frag

In [ ]:
ccd2frag_stats = pd.concat({
    'all': ccds_gm12878.groupby('has_minors').size(),
    'has_fragments': ccd2frag.groupby(['has_minors']).ccd_id.nunique()
}).unstack('has_minors', fill_value=0)
ccd2frag_stats

In [ ]:
scipy.stats.chi2_contingency(
   ccd2frag_stats, correction=True
)

#### GEMs per unit length

In [ ]:
gem_cnt = ccd2frag.groupby(['has_minors'], observed=True).gem_id.nunique()
gem_cnt

In [ ]:
tot_ccd_len = ccds_gm12878.set_index('has_minors', append=True).groupby('has_minors').length.sum()
tot_ccd_len

In [ ]:
gems_per_mb = gem_cnt / (tot_ccd_len / 1e6)  # per 1Mb
print(gems_per_mb[1] / gems_per_mb[0])
gems_per_mb

#### Order of interactions

In [ ]:
arity_stats = ccd2frag.groupby(['n_fragments', 'has_minors']).gem_id.nunique().unstack('has_minors', fill_value=0)
arity_stats.loc[10, :] += arity_stats.loc[11:, :].sum(axis=0)
arity_stats = arity_stats.loc[:10, :]
arity_stats['prop_has_minors'] = arity_stats[True] / (arity_stats[True] + arity_stats[False])
arity_stats

In [ ]:
linked_ccds_multiplex_arity = ccd2frag.loc[ccd2frag.has_minors, :]\
    .groupby(['ccd_id', 'gem_id']).size().to_frame('n_mapped')\
    .groupby(['ccd_id', 'n_mapped']).size()\
    .unstack('n_mapped', fill_value=0)
linked_ccds_multiplex_arity.loc[:, 6] += linked_ccds_multiplex_arity.loc[:, 7:].sum(axis=1)
linked_ccds_multiplex_arity = linked_ccds_multiplex_arity.loc[:, :6]
linked_ccds_multiplex_arity

In [ ]:
(linked_ccds_multiplex_arity.loc[:, 6] > 0).mean()

In [ ]:
seg2frag = segments_pr.join(fragments_pr, suffix='_frag', how='left').df
seg2frag

In [ ]:
mapped_segments = seg2frag.groupby(['Chromosome', 'ccd_id', 'minor_id', 'gem_id'], observed=True).segment_idx.nunique().to_frame('n_mapped').reset_index()
mapped_segments

In [ ]:
mapped_segments.n_mapped.value_counts()

In [ ]:
map_stat = mapped_segments.groupby(['Chromosome', 'ccd_id'], observed=True).n_mapped.max().value_counts().sort_index().to_frame('n_ccds')
map_stat['p_ccds'] = np.round(100.0 * map_stat['n_ccds'] / len(mapped_segments.groupby(['Chromosome', 'ccd_id'], observed=True)), 1)
print(np.cumsum(map_stat.p_ccds.to_numpy()[::-1]))
map_stat

In [ ]:
fully_supported_minors = mapped_segments[mapped_segments['n_mapped'] >= 6]
fully_supported_minors

In [ ]:
ccds_gm12878.reset_index().rename(columns={'chromosome': 'Chromosome'})[['Chromosome', 'ccd_id', 'length', 'n_nodes', 'n_edges']]

In [ ]:
pd.merge(
    fully_supported_minors,
    ccds_gm12878.reset_index().rename(columns={'chromosome': 'Chromosome'})[['Chromosome', 'ccd_id', 'length', 'n_nodes', 'n_edges', 'start', 'end']],
    on=['Chromosome', 'ccd_id'],
    how='left'
)

## Compartments

In [ ]:
raw_compartments = pd.read_csv('./raw_data/hg38_GM12878_compartments_250kb.bedgraph', header=None, sep='\t')
raw_compartments.columns = ['chromosome', 'start', 'end', 'score']
raw_compartments['chromosome'] = raw_compartments['chromosome'].astype(HumanChromosomeDtype)
raw_compartments = raw_compartments.loc[~raw_compartments.score.isna(), :]
raw_compartments['compartment'] = np.where(raw_compartments.score >= 0, 'A', 'B')
print(raw_compartments.groupby('compartment').size())

comp_pr = pr.PyRanges(raw_compartments.rename(columns={'chromosome': 'Chromosome', 'start': 'Start', 'end': 'End'}))
comp_pr

In [ ]:
raw_compartments = pd.read_csv(
    './raw_data/GSE63525_GM12878_subcompartments.bed',
    header=None, sep='\s+', usecols=list(range(4))
).sort_values([0, 1, 2])
raw_compartments.columns = ['chromosome', 'start', 'end', 'subcompartment']
raw_compartments['end'] -= 1  # to prevent spurious overlaps
raw_compartments['chromosome'] = raw_compartments['chromosome'].astype(HumanChromosomeDtype)
raw_compartments['compartment'] = raw_compartments['subcompartment'].str[0]
raw_compartments = raw_compartments.loc[~raw_compartments.compartment.isna(), :]

print(raw_compartments.groupby('compartment').size())
print(raw_compartments.groupby('subcompartment').size())

comp_pr = pr.PyRanges(raw_compartments.rename(columns={'chromosome': 'Chromosome', 'start': 'Start', 'end': 'End'}))
comp_pr

In [ ]:
ccd2comp = ccds_pr.join(comp_pr, suffix='_comp', report_overlap=True).df.set_index(['Chromosome', 'ccd_id'])
ccd2comp

In [ ]:
_overlap_by_comp = ccd2comp.groupby(['has_minors', 'Chromosome', 'ccd_id', 'compartment'], observed=False)\
    .Overlap.sum().unstack('compartment', fill_value=0)
coverage = ccds_gm12878.reset_index()\
    .rename(columns={'chromosome': 'Chromosome'})\
    .set_index(['has_minors', 'Chromosome', 'ccd_id'])\
    .loc[:, ['length']]\
    .join(_overlap_by_comp)
coverage

In [ ]:
coverage['A'] /= coverage['length']
coverage['B'] /= coverage['length']
coverage

In [ ]:
coverage.groupby('has_minors').mean()

In [ ]:
coverage['compartment'] = 'mixed'
coverage.loc[coverage['A'] >= 0.6, 'compartment'] = 'A'
coverage.loc[coverage['B'] >= 0.6, 'compartment'] = 'B'
tab_comp = coverage.groupby('has_minors').compartment.value_counts().to_frame('n_ccds')
tab_comp['p_ccds'] = 100.0 * tab_comp['n_ccds'] / tab_comp.groupby('has_minors').n_ccds.sum()
tab_comp

In [ ]:
_cont_table = tab_comp.n_ccds.unstack('compartment', fill_value=0)
print(_cont_table)
scipy.stats.chi2_contingency(_cont_table, correction=True)

In [ ]:
p = tab_comp.p_ccds
print(p.loc[False, 'B'] / p.loc[False, 'A'])
print(p.loc[True, 'B'] / p.loc[True, 'A'])

In [ ]:
coverage['has_boundary'] = (coverage['A'] > 0.0) & (coverage['B'] > 0.0)
tab_bound = coverage.groupby('has_minors').has_boundary.value_counts().to_frame('n_ccds')
tab_bound['p_ccds'] = 100.0 * tab_bound['n_ccds'] / tab_bound.groupby('has_minors').n_ccds.sum()
tab_bound

In [ ]:
_cont_table = tab_bound.n_ccds.unstack('has_boundary', fill_value=0)
print(_cont_table)
scipy.stats.chi2_contingency(_cont_table, correction=True)

## Figure 4

In [ ]:
def compare_dist_linked_to_nonlinked(X, var, xlab, yticks=False, palette='colorblind', scatter=False, ax=None, xlim=(0, None), log=False, legend=False):
    if ax is None:
        ax = plt.gca()
    else:
        plt.sca(ax)
    hue_order = ['non-linked', 'linked']
    sns.boxplot(
        X, x=var, y="dataset", hue="has_minors_str",
        whis=[0, 100], palette=palette, width=0.8,
        ax=ax, linewidth=0.75, log_scale=log, hue_order=hue_order,
        legend=legend
    )
    if scatter:
        ptpal = sns.color_palette(['white', 'white'], as_cmap=True)
        # ptpal = sns.color_palette(['gray', 'gray'], as_cmap=True)
        # ptpal = 'tab10'
        sns.stripplot(
            X, x=var, y="dataset", hue="has_minors_str",
            size=1, alpha=.2, palette=ptpal, dodge=True,
            linewidth=0.25, edgecolor='auto', log_scale=log,
            ax=ax, legend=False, hue_order=hue_order
        )
    if xlim is not None:
        ax.set_xlim(xlim)
    ax.xaxis.grid(True)
    ax.set(xlabel=xlab)
    ax.set(ylabel="")
    if not yticks:
        plt.tick_params(labelleft=False, left=False)
    if legend:
        plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0., title='')
    sns.despine(trim=False, left=True)    

ccd_stats = ccds.reset_index()
ccd_stats['dataset_str'] = ccd_stats.dataset.astype('str')
ccd_stats['has_minors_str'] = ccd_stats.has_minors.replace({True: 'linked', False: 'non-linked'})
print(ccd_stats.dataset.value_counts())

_plt = functools.partial(compare_dist_linked_to_nonlinked, ccd_stats, scatter=False)

def _lab(s, x=-0.1, y=1.15):
    ax = plt.gca()
    ax.text(x, y, s, horizontalalignment='center', verticalalignment='center', transform=ax.transAxes, fontsize=14)

matplotlib.rcParams.update({'font.size': 8})

fig, axs = plt.subplots(
    nrows=2, ncols=4, figsize=(7.8, 3.8), dpi=300,
    sharex=False, sharey=False
)
iax = iter(axs.flatten())
ilab = iter(string.ascii_uppercase)

# A-F
_plt('n_nodes', 'No. Nodes', log=True, xlim=(1, None), ax=next(iax), yticks=True); _lab(next(ilab), -.5)
_plt('n_edges', 'No. Edges', log=True, xlim=(1, None), ax=next(iax)); _lab(next(ilab))
_plt('density', 'Density', log=True, xlim=(1e-5, 1), ax=next(iax)); _lab(next(ilab))
_plt('degree_mean', 'Avg. Degree', ax=next(iax), legend=True); _lab(next(ilab))
_plt('closeness_mean', 'Avg. Closeness', ax=next(iax), yticks=True); _lab(next(ilab), -.5)
_plt('betweenness_mean', 'Avg. Betweenness', ax=next(iax)); _lab(next(ilab))

# G
plt.sca(next(iax))
sns.barplot(
    x=map_stat.index, y='n_ccds', data=map_stat.reset_index(),
    color='tan', linewidth=.5, edgecolor="black"
)
_lab(next(ilab))
plt.xlabel('Highest multiplex\ninteraction order')
plt.ylabel('No. of CCDs')

# H
plt.sca(next(iax))
tmp = tab_comp.reset_index()
tmp['has_minors'] = tmp['has_minors'].replace({False: 'non-linked', True: 'linked'})
tmp = tmp.set_index(['has_minors', 'compartment'])
tmp.p_ccds.unstack('compartment').plot(
    kind='bar', stacked=True, ax=plt.gca(), color=sns.color_palette('Set2', 3),
    linewidth=.5, edgecolor='black'
)
_lab(next(ilab))
plt.ylim(0, 100)
plt.xlabel('')
plt.xticks(rotation=0)
plt.ylabel('% CCDs')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

# next(iax).axis('off')
plt.subplots_adjust(hspace=0.7, wspace=0.5)

plt.savefig(f'./plots/Figure_4_plots.svg', bbox_inches='tight')
# plt.savefig(f'./plots/Figure_4_plots.png', bbox_inches='tight')
# plt.show()